In [3]:
# On utilise les librairies tabular (données structurées) et collab pour le filtrage collaboratif. 
from fastai2.collab import *
from fastai2.tabular.all import *

In [6]:
# FastAi propose un jeux de données du site Movie Lens. C'est un sous ensemble de 100.000 notes par des utilisateurs. 
path = untar_data(URLs.ML_100k)

In [9]:
# On charge les données du fichier csv fourni pour voir à quoi elles ressemblent. 
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None, names=['user','movie','rating','timestamp'])
# On affiche les 10 premières lignes
ratings.head(10)

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [11]:
# On récupére le nom des films pour les id fournis dans le fichier précédent. 
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1', 
                     usecols=(0,1), names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [12]:
# La librairie panda permet de merge les données basé sur le nom de la colonne
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [14]:
# FastAI propose des DataLoader spécifiques (Comportement par défaut pour filtrage collaboratif)
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,807,"Crow, The (1994)",4
1,542,"Clockwork Orange, A (1971)",4
2,116,Scream (1996),3
3,102,G.I. Jane (1997),3
4,426,Ben-Hur (1959),4
5,727,Escape from L.A. (1996),3
6,585,"Magnificent Seven, The (1954)",5
7,152,Babe (1995),5
8,693,"Substitute, The (1996)",3
9,683,Mimic (1997),2


In [15]:
# On récupére le nombre d'utilisateur et de films contenus dans notre jeux de données
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

In [16]:
# Le principe que l'on va appliqué correspond aux "caractéristiques latentes". l'idées est d'attribuer x caractéristiques 
# aléatoires au éléments (film et user ici). Ces caractèristiques correspondraient à des types de films (SF, Romantique, 
# ...). On va ensuite grace aux machine learning essayé de déterminer ses caractèristiques pour chaque élément en fonction
# des notes attribuées. 
# Exemple: User1 [0.1, 0.9] (Aime pas trop la SF mais bien le Romantique)
#          Film1 [0.9, 0.2], Film2 [0.1, 0.8]
# Si je multiplie les caractérisque de mon utilisateur à celle des films 0.1*0.9 + 0.9*0.2 et 0.1*0.1 + 0.9*0.8 j'obtiens 
# un résultat nettement supérieur pour le second film (il est romantique et l'utilisateur aime les film romantique)
# Dans les fait les "caractéristiques latentes" ne sont pas définies, on ne sait pas à quoi elles correspondent. 

# On initialise donc 5 caractéristiques aléatoirement pour les utilisateurs et les films

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)
user_factors[0]


tensor([ 0.0212, -1.2224, -0.4576,  0.9599, -0.4317])